In [ ]:
{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"provenance":[],"gpuType":"T4"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"},"accelerator":"GPU"},"cells":[{"cell_type":"code","execution_count":null,"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"g1N7PPF6ujXE","executionInfo":{"status":"ok","timestamp":1740659728429,"user_tz":-330,"elapsed":142,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"f30c3706-7b2c-4630-8541-97f80ad41cc2"},"outputs":[{"output_type":"stream","name":"stdout","text":["Thu Feb 27 12:35:28 2025       \n","+-----------------------------------------------------------------------------------------+\n","| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |\n","|-----------------------------------------+------------------------+----------------------+\n","| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |\n","| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |\n","|                                         |                        |               MIG M. |\n","|=========================================+========================+======================|\n","|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |\n","| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |\n","|                                         |                        |                  N/A |\n","+-----------------------------------------+------------------------+----------------------+\n","                                                                                         \n","+-----------------------------------------------------------------------------------------+\n","| Processes:                                                                              |\n","|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |\n","|        ID   ID                                                               Usage      |\n","|=========================================================================================|\n","|  No running processes found                                                             |\n","+-----------------------------------------------------------------------------------------+\n"]}],"source":["!nvidia-smi"]},{"cell_type":"code","source":["%%writefile addtion_on_device.cu\n","\n","#include<stdio.h>\n","__global__ void add(int *a, int *b, int *c) {\n","\t\t*c = *a + *b;\n","}\n","int main(void) {\n","\t\tint a, b, c;\t            // host copies of a, b, c\n","\t\tint *d_a, *d_b, *d_c;\t     // device copies of a, b, c\n","\t\tint size = sizeof(int);\n","\n","\t\t// Allocate space for device copies of a, b, c\n","\t\tcudaMalloc((void **)&d_a, size);\n","\t\tcudaMalloc((void **)&d_b, size);\n","\t\tcudaMalloc((void **)&d_c, size);\n","\n","\t\t// Setup input values\n","\t\ta = 2;\n","\t\tb = 7;\n","\t\t// Copy inputs to device\n","\t\tcudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);\n","\t\tcudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);\n","\n","\t\t// Launch add() kernel on GPU\n","\t\tadd<<<1,1>>>(d_a, d_b, d_c);\n","\n","\t\t// Copy result back to host\n","\t\tcudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);\n","\n","\t\t// Cleanup\n","\t\tcudaFree(d_a); cudaFree(d_b); cudaFree(d_c);\n","    printf(\"Value of c after device addtion is %d\",c);\n","\t\treturn 0;\n","\t}"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"gVyY5bxcwv7x","executionInfo":{"status":"ok","timestamp":1740659744473,"user_tz":-330,"elapsed":48,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"5aed4969-3f9c-4848-8c21-2bbb1c7264d4"},"execution_count":null,"outputs":[{"output_type":"stream","name":"stdout","text":["Writing addtion_on_device.cu\n"]}]},{"cell_type":"code","source":["!nvcc -arch=sm_75 addtion_on_device.cu -o aod"],"metadata":{"id":"DK8TKS5qw0VX"},"execution_count":null,"outputs":[]},{"cell_type":"code","source":["!./aod"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"As8tIfvex9oR","executionInfo":{"status":"ok","timestamp":1740659786607,"user_tz":-330,"elapsed":484,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"4d37e66d-7c8b-4de4-da2e-77f97b648fdd"},"execution_count":null,"outputs":[{"output_type":"stream","name":"stdout","text":["Value of c after device addtion is 9"]}]},{"cell_type":"code","source":["%%writefile P_AOD.cu\n","\n","#include<stdio.h>\n","#define N 512\n","__global__ void add(int *a, int *b, int *c) {\n","\t\tc[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];\n","}\n","int main(void) {\n","\tint *a, *b, *c;\t\t// host copies of a, b, c\n","\tint *d_a, *d_b, *d_c;\t// device copies of a, b, c\n","\tint size = N * sizeof(int);\n","\n","\t// Alloc space for device copies of a, b, c\n","\tcudaMalloc((void **)&d_a, size);\n","\tcudaMalloc((void **)&d_b, size);\n","\tcudaMalloc((void **)&d_c, size);\n","\n","\t// Alloc space for host copies of a, b, c and setup input values\n","\ta = (int *)malloc(size);\n","\tb = (int *)malloc(size);\n","\tc = (int *)malloc(size);\n","  int counter;\n","  for(counter = 0; counter < N; counter++){\n","    a[counter] = counter;\n","    b[counter] = counter*2;\n","  }\n","        // Copy inputs to device\n","        cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);\n","        cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);\n","\n","        // Launch add() kernel on GPU with N blocks\n","        add<<<N,1>>>(d_a, d_b, d_c);\n","\n","        // Copy result back to host\n","        cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);\n","\n","        // Cleanup\n","        free(a); free(b); free(c);\n","        cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);\n","        for(counter = 0; counter < 10; counter++){\n","          printf(\"\\nValue of c[%d] = %d\",counter,c[counter]);\n","          b[counter] = counter*2;\n","        }\n","        return 0;\n","  }"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"bSli66lKBHlS","executionInfo":{"status":"ok","timestamp":1740659798123,"user_tz":-330,"elapsed":14,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"51686686-e029-4296-e9ff-ed086315b82c"},"execution_count":null,"outputs":[{"output_type":"stream","name":"stdout","text":["Writing P_AOD.cu\n"]}]},{"cell_type":"code","source":["!nvcc -arch=sm_75 P_AOD.cu -o p_aod"],"metadata":{"id":"-vK8YFyMCyd0"},"execution_count":null,"outputs":[]},{"cell_type":"code","source":["!./p_aod"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"gb-EEUPeFMcm","executionInfo":{"status":"ok","timestamp":1740659844519,"user_tz":-330,"elapsed":301,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"966bf640-291d-4835-95d1-baed0fe1e9eb"},"execution_count":null,"outputs":[{"output_type":"stream","name":"stdout","text":["\n","Value of c[0] = 0\n","Value of c[1] = 3\n","Value of c[2] = 6\n","Value of c[3] = 9\n","Value of c[4] = 12\n","Value of c[5] = 15\n","Value of c[6] = 18\n","Value of c[7] = 21\n","Value of c[8] = 24\n","Value of c[9] = 27"]}]}]}